In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

Saving Untitled document.txt to Untitled document (1).txt


OSError: ignored

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 11107 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

﻿"Look at this stupid dog." Ramesh exclaimed
exasperated. "He's scared of his own shadow." 
"What'd he do this time?" asked Ayesha. Ayesha and Ramesh were a happy couple who had been married for nine years. They lived in a medium-size ranch house i


In [ ]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

In [ ]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

In [ ]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to 

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
﻿"Look at this stupid dog." Ramesh exclaimed
exasperated. "He's scared of his own shadow." 
"What'

OUTPUT
"Look at this stupid dog." Ramesh exclaimed
exasperated. "He's scared of his own shadow." 
"What'd


EXAMPLE

INPUT
 he do this time?" asked Ayesha. Ayesha and Ramesh were a happy couple who had been married for nine

OUTPUT
he do this time?" asked Ayesha. Ayesha and Ramesh were a happy couple who had been married for nine 


In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           15360     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 60)            61500     
Total params: 5,323,836
Trainable params: 5,323,836
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  #

(64, 100, 60) # (batch_size, sequence_length, vocab_size)


In [ ]:
def losts(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=losts)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
import time
x = time.time()
history = model.fit(data, epochs=5000, callbacks=[checkpoint_callback])
answer = time.time()-x

Epoch 1/5000
1/1 [==============================] - 0s 114ms/step - loss: 4.0924
Epoch 2/5000
1/1 [==============================] - 0s 91ms/step - loss: 4.0365
Epoch 3/5000
1/1 [==============================] - 0s 166ms/step - loss: 3.8132
Epoch 4/5000
1/1 [==============================] - 0s 126ms/step - loss: 3.6463
Epoch 5/5000
1/1 [==============================] - 0s 129ms/step - loss: 3.6564
Epoch 6/5000
1/1 [==============================] - 0s 121ms/step - loss: 3.5710
Epoch 7/5000
1/1 [==============================] - 0s 158ms/step - loss: 3.3564
Epoch 8/5000
1/1 [==============================] - 0s 122ms/step - loss: 3.2224
Epoch 9/5000
1/1 [==============================] - 0s 139ms/step - loss: 3.1901
Epoch 10/5000
1/1 [==============================] - 0s 130ms/step - loss: 3.2491
Epoch 11/5000
1/1 [==============================] - 0s 179ms/step - loss: 3.1419
Epoch 12/5000
1/1 [==============================] - 0s 140ms/step - loss: 3.1369
Epoch 13/5000
1/1 [=======

ResourceExhaustedError: ignored